In [2]:
% matplotlib inline
import numpy as np 
import os 
import cv2
from random import shuffle
from tqdm import tqdm
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [3]:
train_dir = "train"
test_dir = "test1"

img_size = 50
lr = 1e-3

MODEL_NAME = "dog_cat_{}_{}.model".format(lr,"2convbasic")

In [4]:
def label_img(img):
    word_label = img.split('.')[-3]

    if word_label == "cat":
        return [1,0]
    if word_label == "dog":
        return [0,1]

In [5]:
def create_train_data():
	training_data=[]
	for img in tqdm(os.listdir(train_dir)):
		label = label_img(img)
		img_path = os.path.join(train_dir,img)
		img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
		img = cv2.resize(img, (img_size,img_size))
		training_data.append([np.array(img),np.array(label)])

	shuffle(training_data)
	np.save('training_data.npy',training_data)

	return training_data

In [6]:
def create_test_data():
	testing_data=[]
	for img in tqdm(os.listdir(test_dir)):
		img_path = os.path.join(test_dir,img)
		img_num = img.split('.')[0]
		img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
		img = cv2.resize(img, (img_size,img_size))
		testing_data.append([np.array(img), img_num])

	shuffle(testing_data)
	np.save('testing_data.npy',testing_data)

	return testing_data

In [7]:
# train_data = create_train_data()

train_data = np.load('training_data.npy')

In [8]:
convnet = input_data(shape=[None,img_size,img_size,1],name = 'input')

convnet = conv_2d(convnet,32,5,activation = 'relu')
convnet = max_pool_2d(convnet,5)

convnet = conv_2d(convnet,64,5,activation='relu')
convnet = max_pool_2d(convnet,5)

convnet = fully_connected(convnet,1024,activation='relu')
convnet = dropout(convnet,0.7)

convnet = fully_connected(convnet,2,activation = 'softmax')
convnet = regression(convnet, optimizer = 'adam', learning_rate = lr, loss = 'categorical_crossentropy', name = 'targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [9]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

In [10]:
train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,img_size,img_size,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,img_size,img_size,1)
test_y = [i[1] for i in test]

In [10]:
print(model.fit({'input':X},{'targets':Y},n_epoch = 3,validation_set=({'input':test_x}, {'targets':test_y}), snapshot_step=500,show_metric = True, run_id = MODEL_NAME))


Training Step: 1148  | total loss: 11.41822 | time: 109.012s
| Adam | epoch: 003 | loss: 11.41822 - acc: 0.5041 -- iter: 24448/24500
Training Step: 1149  | total loss: 11.57160 | time: 110.298s
| Adam | epoch: 003 | loss: 11.57160 - acc: 0.4975 | val_loss: 10.86820 - val_acc: 0.5280 -- iter: 24500/24500
--
None


In [14]:
pred = model.predict(test_x[:50])

In [ ]:
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image',test[0][1])
cv2.waitKey(0)
cv2.destroyAllWindows()